# Recursive Feature Elimination (RFE) - Deployment

This component performs feature ranking with recursive feature elimination based on a Random Forest estimator with default hyperparameters. K-fold cross-validation is employed to estimate feature importance. It uses the `RFECV` implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load a model and other artifacts.
- how to use a model to provide real-time transformations.

## Wrapping Model

Allows your component to expose a service over REST.

To wrap your model [follow the instructions](https://docs.seldon.io/projects/seldon-core/en/v0.3.0/python/python_component.html) for your chosen language or toolkit.

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
from platiagro import load_model

logger = logging.getLogger(__name__)


class Model(object):
    def __init__(self, dataset: str = None, target: str = None,):
        logger.info(f"dataset: {dataset}")
        logger.info(f"target: {target}")

        # Load Artifacts: Estimator, etc
        model = load_model()
        self.pipeline = model["pipeline"]
        self.selected_features = model["selected_features"]

    def class_names(self):
        return self.selected_features

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        return self.pipeline.transform(X)

## API Contract

There are two sections:

- `features` : The feature array you intend to send in a request
- `targets` : The response you expect back

Each section has a list of definitions. Each definition consists of:

- `name` : String : The name of the feature
- `ftype` : one of CONTINUOUS, CATEGORICAL : the type of the feature
- `dtype` : One of FLOAT, INT : Required for ftype CONTINUOUS : What type of feature to create
- `values` : list of Strings : Required for ftype CATEGORICAL : The possible categorical values
- `range` : list of two numbers : Optional for ftype CONTINUOUS : The range of values (inclusive) that a continuous value can take
- `repeat` : integer : Optional value for how many times to repeat this value
- `shape` : array of integers : Optional value for the shape of array to coerce the values

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "crim",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "zn",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "indus",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "chas",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "nox",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1.0]
        },
        {
            "name": "rm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 10.0]
        },
        {
            "name": "age",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "dis",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "rad",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 24.0]
        },
        {
            "name": "tax",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "ptratio",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        },
        {
            "name": "black",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 1000.0]
        },
        {
            "name": "lstat",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ],
    "targets": [
        {
            "name": "estimate",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.0, 100.0]
        }
    ]
}

## Test Deployment

Starts a service wrapping a feature selector, sends a request to the service, and shows the response.

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")